In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from requests_html import HTMLSession
import numpy as np
import re

# School URLs

In [ ]:
school_list = []

rge = range(0,10)
d = {i:i for i in rge}
df = pd.DataFrame(columns=['url'])
for j in range(1, 184):
    html = requests.get('https://premium.usnews.com/best-colleges/search?_sort=rank&_sortDirection=asc&_page={}'.format(j), headers = {
        "Host": "www.usnews.com",
        "Connection": "keep-alive",
        "Cache-Control": "max-age=0",
        "Upgrade-Insecure-Requests": "1",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN,zh;q=0.9",
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"
    }).text
    soup = BeautifulSoup(html, 'lxml')    
    
    for i in d:
        li = soup.find('li', id = 'school-%s'%i)
        
        if not li:
            break
        
        h3 = li.h3
        url = h3.a['href']
        
        school_list.append(url)
        
        
data = {
    "school": school_list
}

url_list = pd.DataFrame(data)
pd.set_option('display.max_rows', 2000)
url_list

url_list.to_csv('material/urls.csv')



# Student Life

In [ ]:
Urls = pd.read_csv('material/urls.csv', names=['University', 'URL'])

regular_info = pd.DataFrame(index=Urls['University'])
pre_col = ['description', 'Mission & Unique Qualities',
           'Types of campus housing available', 'Collegiate athletic association', 'Campus Safety']
for i in pre_col:
    regular_info[i] = 'N/A'


for i in range( len(Urls)):
    url = ('https://www.usnews.com' + Urls.iloc[i, 1] + '/student-life').replace(' ', '')
    print(url)

    session = HTMLSession()
    page = session.get(url, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')


    # description
    description = list(soup.select('#summary-box > react-trigger > div > p'))[0].text
    regular_info['description'][i] = description


    # life stats
    life_stats = soup.find(class_='quick-stat-box')
    for j in life_stats.find_all('div'):
        category = j.find('dt').text
        if category not in regular_info.columns:
            regular_info[category] = 'N/A'
        regular_info[category][i] = j.find('dd').text


    # body
    body = list(soup.select('#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.izoLfu.mb0 > div > div > div:nth-child(2) > div'))[0]

    # School Mission & Unique Qualities
    MUQ = list(body.find_all('div', recursive=False))[0].find('div')
    regular_info['Mission & Unique Qualities'][i] = MUQ.text.replace('Content is provided by the school.', '')

    # Student Body
    student_body = list(body.find_all('div', recursive=False))[1].find('div').find_all('div', recursive=False)
    for j in student_body:
        if len(j.find_all('p')) == 2:
            if j.find_all('p')[0].text.strip() not in regular_info.columns:
                regular_info[j.find_all('p')[0].text.strip()] = 'N/A'
            regular_info[j.find_all('p')[0].text.strip()][i] = j.find_all('p')[1].text
        else:
            gender = j.find('div').find_all('div', recursive=False)[1].find_all('div')
            for k in gender:
                res = k.find('b').text
                title = k.text.replace(res, '')
                if title not in regular_info.columns:
                    regular_info[title] = 'N/A'
                regular_info[title][i] = res

    # Housing & Dorms at Harvard University
    housing_dorms = list(body.find_all('div', recursive=False))[2].find('div')
    if len(list(housing_dorms)[0].find_all('p')) == 2:
        j = list(housing_dorms)[0]
        if j.find_all('p')[0].text.strip() not in regular_info.columns:
            regular_info[j.find_all('p')[0].text.strip()] = 'N/A'
        regular_info[j.find_all('p')[0].text.strip()][i] = j.find_all('p')[1].text
    else:
        live = list(housing_dorms)[0].find('div').find_all('div', recursive=False)[1].find_all('div')
        for k in live:
            res = k.find('b').text
            title = k.text.replace(res, '')
            if title not in regular_info.columns:
                regular_info[title] = 'N/A'
            regular_info[title][i] = res

    if len(list(housing_dorms)[1].find_all('p')) == 2:
        regular_info['Types of campus housing available'][i] = list(housing_dorms)[1].find_all('p')[0].text
    else:
        types = list(housing_dorms)[1].find('div').find_all('li')
        res = []
        for k in types:
            res.append(k.text)
        res = '， '.join(res)
        regular_info['Types of campus housing available'][i] = res

    # sports
    sports = list(body.find_all('div', recursive=False))[3]
    regular_info['Collegiate athletic association'][i] = sports.div.div.find_all('p')[1].text

    # Clubs & Organizations
    clubs_organizations = list(body.find_all('div', recursive=False))[4].find('div', recursive=False)
    for j in clubs_organizations:
        if len(j.find_all('p')) == 2:
            title = j.find_all('p')[0].text
            res = j.find_all('p')[1].text
            if title not in regular_info.columns:
                regular_info[title] = 'N/A'
            regular_info[title][i] = res
        else:
            gender = j.find('p').text
            classify = j.find('div').find_all('div', recursive=False)[1].find_all('div')
            for k in classify:
                res = k.find('b').text
                title = k.text.replace(res, '')
                if title not in regular_info.columns:
                    regular_info[gender + title] = 'N/A'
                regular_info[gender + title][i] = res


    # safety
    safety = list(soup.select('#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.izoLfu.mb0 > div > div > react-trigger > div > div.section-box'))[0]
    regular_info['Campus Safety'][i] = safety.find('p').text


    # # criminal
    # criminal = list(soup.select('#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.izoLfu.mb0 > div > div > react-trigger > div > react-trigger'))[0]
    # for j in criminal:
    # 	pass

In [ ]:
# drop columns that are full of 'N/A'
for i in range(len(regular_info.columns)):
    if len(set(regular_info.iloc[:, i])) == 1:
        print(i)
        print(regular_info.columns[i])

regular_info = regular_info.drop(['Students living on- and off-campus', 'Degree-seeking student gender distribution',
                                  'Undergraduate men who are members of a fraternity',
                                  'Undergraduate women who are members of a sorority'], axis=1)


# roughly clean the data
def clean(column):
    for m in range(len(regular_info)):
        temp = regular_info[column][m].split('\n')
        for n in range(len(temp)):
            temp[n] = temp[n].strip()
        temp = ' '.join(temp)
        regular_info[column][m] = temp


clean('description')
clean('Campus Safety')


# regular_info.to_csv('../../user-notebooks/yjin/USNews/results/student life.csv')


# Ranking

get categories and other prepare

In [ ]:
Urls = pd.read_csv('material/urls.csv', names=['University', 'URL'])
headers = {
    "Host": "www.usnews.com",
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "Upgrade-Insecure-Requests": "1",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh;q=0.9",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"
}

url = "https://www.usnews.com/best-colleges/search"
proxy = "127.0.0.1:8866"
response = requests.get(url, headers=headers)
data = response.text
soup = BeautifulSoup(data, 'html.parser')
sel = soup.find_all("select", id='schoolType')
school_type_options = list(sel)[0].find_all("option")
school_type_options_text = [i.text for i in school_type_options][1:]
school_type_options_value = [i.get('value') for i in school_type_options][1:]


def get_ranks(lin):
    out = []
    for i in range(1, 100):
        resp = requests.get(lin + '?_page=' + str(i), headers=headers)
        if resp.status_code == 404:
            break

        s = BeautifulSoup(resp.text, 'lxml')
        for j in range(0, 10):
            l_i = s.find('li', id='school-' + str(j))
            if not l_i:
                break
            out.append(l_i.h3.a.text)
    return out

ranking_df = pd.DataFrame(index=Urls['University'])

scrape and store results to csv table

In [ ]:
for st in school_type_options_value:
    link = 'https://www.usnews.com/best-colleges/rankings/{0}'.format(st)
    if st == 'business':
        link = link + '-overall'
    school_name = get_ranks(link)
    ranking_df[st.replace('-', ' ')] = 0
    for i in range(len(school_name)):
        ranking_df.loc[school_name[i], st.replace('-', ' ')] = i

    soup = BeautifulSoup(requests.get(link, headers=headers).text, 'html.parser')
    ranking_options = soup.find_all("select", id='ranking')[0].find_all("option")
    ranking_options_text = [i.text for i in ranking_options][1:]
    ranking_options_value = [i.get('value') for i in ranking_options][1:]


    for r in ranking_options_value:
        if '-' in st:
            link = 'https://www.usnews.com/best-colleges/rankings/{0}/{1}'.format(st, r)
        else:
            link = 'https://www.usnews.com/best-colleges/rankings/{0}-{1}'.format(st, r)
        school_name = get_ranks(link)

        col_name = st.replace('-', ' ') + ' - ' + r.replace('-', ' ')
        ranking_df[col_name] = 0
        for i in range(len(school_name)):
            ranking_df.loc[school_name[i], col_name] = i + 1

# ranking_df.to_csv('results/ranking.csv')

## Get USNews ID

In [ ]:
Urls = pd.read_csv('material/urls.csv', header=None)
Urls.columns = ['University', 'Url']
Urls['USNews_ID'] = [i[-4:] for i in urls.iloc[:, 1]]
# Urls.to_csv('../../user-notebooks/yjin/USNews/results/USNewsID.csv')

# Tuition and Financial Aid

## scrape general tables

In [ ]:
content_df = pd.DataFrame(columns=["University", "Content"])
texts = []

for i in range(len(Urls)):
    url = ('https://www.usnews.com' + Urls.iloc[i, 1] + '/paying').replace(' ', '')
    session = HTMLSession()
    content = session.get(url)
    basic_address = "#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > " \
                    "div > div > div:nth-child(2) > div"

    # Tuition & Expenses
    temp_sector = basic_address + " > div:nth-child(1) > div:nth-child(2)"
    results = content.html.find(temp_sector)
    text = results[0].text
    texts.append(text)

content_df['University'] = Urls['University']
content_df['Content'] = texts

## format the tables

In [ ]:
data = content_df
average_annual_cost_by_family_income = pd.DataFrame(columns=['$0-$30,000', '$30,001-$48,000', '$48,001-$75,000',
                                                             '$75,001-$110,000', '$110,001+'], index=data['University'])
general_cost = pd.DataFrame(columns=['Tuition and fees', 'In-state tuition and fees', 'Out-of-state tuition and fees',
                                     'Average annual cost', 'Room and board'], index=data['University'])

for i in range(len(data)):
    check = re.findall('\n\(\d\d\d\d-\d\d\)', data.iloc[i, 1])
    for element in set(check):
        data.iloc[i, 1] = data.iloc[i, 1].replace(element, '')
    temp = np.array(data.iloc[i, 1].split('\n'))
    help_ind = np.where(temp == 'Average annual cost by family income')[0][0]
    j = 0
    temp = np.array(data.iloc[i, 1].split('\n'))
    while True:
        if j == help_ind:
            average_annual_cost_by_family_income.iloc[i, :] = temp[[k + j for k in [14, 16, 18, 20, 22]]]
            j += 23
        elif temp[j] != "Go to this school's net price calculator":
            general_cost[temp[j]][i] = temp[j + 1]
            j += 2
        else:
            break

general_cost = general_cost.fillna('N/A')
average_annual_cost_by_family_income = average_annual_cost_by_family_income.fillna('N/A')

# average_annual_cost_by_family_income.to_csv('../../user-notebooks/yjin/USNews/results/average_annual_cost_by_family_income.csv')
# general_cost.to_csv('../../user-notebooks/yjin/USNews/results/general_cost.csv')

## scrape quick status in Tuition and Financial Aid block

In [ ]:
quick_stats_df = pd.DataFrame(index=Urls['University'])

for i in range(len(Urls)):
    url = ('https://www.usnews.com' + Urls.iloc[i, 1] + '/paying').replace(' ', '')
    session = HTMLSession()
    content = session.get(url)
    basic_address = "#app > div > div:nth-child(1) > div.Heading__ProfileHeadingBox-sc-6a32fq-5.fafMtc > div > div > " \
                    "div > div.Villain__FlexDiv-sc-1y12ps5-0.ckPKDS > div.Villain__SupplementColumn-sc-1y12ps5-2.kKLGsQ > " \
                    "div > div > div"
    results_0 = content.html.find(basic_address)
    if len(results_0) > 0:
        text = results_0[0].text
        n = text.count('\n', 0, len(text))
        for j in range(1, n + 1):
            item_path = basic_address + '> div.Panel__Content-ejd83f-0.ftmBcb.border-right.border-left.border-bottom > ' \
                                        'table > tbody > tr:nth-child({0}) > div > a.Anchor-byh49a-0.iPzQVi'.format(j)
            temp_item = content.html.find(item_path)[0].text
            value_path = item_path.replace('a.Anchor-byh49a-0.iPzQVi', 'a.Anchor-byh49a-0.gbsGda')
            temp_value = content.html.find(value_path)[0].text
            if temp_item in quick_stats_df.columns:
                quick_stats_df[temp_item][i] = temp_value
            else:
                quick_stats_df[temp_item] = 'N/A'
                quick_stats_df[temp_item][i] = temp_value

# quick_stats_df.to_csv('../../user-notebooks/yjin/USNews/results\FA quick stats.csv')

## scrape the student loan block

### scrape the text part (student loan)

In [ ]:
student_loan = pd.DataFrame(columns=["University"], index=Urls['University'])
texts = []


for i in range(len(Urls)):
    url = ('https://www.usnews.com' + Urls.iloc[i, 1] + '/paying').replace(' ', '')
    # print(url)
    session = HTMLSession()
    content = session.get(url)
    basic_address = "#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > div > " \
                    "div > div:nth-child(2) > div > div:nth-child(3) > div:nth-child(3)"
    results_0 = content.html.find(basic_address)
    if len(results_0) > 0:
        text = results_0[0].text
        texts.append(text)
    else:
        texts.append('N/A')

student_loan['Content'] = texts
# student_loan.to_csv('../../user-notebooks/yjin/USNews/data\student loan text.csv')

### scrape the tables (student loan）

In [ ]:
Typical_total_federal_loan_debt_by_family_income = pd.DataFrame(index=Urls['University'])
Undergraduates_federal_loan_debt_by_family_income = pd.DataFrame(index=Urls['University'])
general_df = pd.DataFrame(index=Urls['University'])

for i in range(len(Urls)):
    url = ('https://www.usnews.com' + Urls.iloc[i, 1] + '/paying').replace(' ', '')
    print(url)
    session = HTMLSession()
    content = session.get(url)
    basic_address = "#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > div > " \
                    "div > div:nth-child(2) > div > div:nth-child(3) > div:nth-child(3)"
    j = 1
    # not sure how much sub blocks are there so use while loop
    while True:
        temp_path = basic_address + '> div:nth-child({0})'.format(j)
        results_0 = content.html.find(temp_path)
        # if the block exists
        if len(results_0) > 0:
            text = results_0[0].text
            temp = text.split('\n')

            # if not the sub tables
            if len(temp) == 2:
                # if column exists, and if not
                if temp[0] in general_df.columns:
                    general_df[temp[0]][i] = temp[1]
                else:
                    general_df[temp[0]] = 'N/A'
                    general_df[temp[0]][i] = temp[1]

            # see which sub table it belongs to
            else:
                n = int((len(temp)-3)/4)
                if temp[0] == 'Typical total federal loan debt by family income':
                    for k in range(n):
                        if temp[2*(k+n) + 3] in Typical_total_federal_loan_debt_by_family_income.columns:
                            Typical_total_federal_loan_debt_by_family_income[temp[2*(k+n) + 3]][i] = temp[2*(k+n) + 4]
                        else:
                            Typical_total_federal_loan_debt_by_family_income[temp[2*(k+n) + 3]] = 'N/A'
                            Typical_total_federal_loan_debt_by_family_income[temp[2*(k+n) + 3]][i] = temp[2*(k+n) + 4]
                elif temp[0] == 'Undergraduates paying down their federal loan debt by family income':
                    for k in range(n):
                        if temp[2*(k+n) + 3] in Undergraduates_federal_loan_debt_by_family_income.columns:
                            Undergraduates_federal_loan_debt_by_family_income[temp[2*(k+n) + 3]][i] = temp[2*(k+n) + 4]
                        else:
                            Undergraduates_federal_loan_debt_by_family_income[temp[2*(k+n) + 3]] = 'N/A'
                            Undergraduates_federal_loan_debt_by_family_income[temp[2*(k+n) + 3]][i] = temp[2*(k+n) + 4]
        else:
            break
        j += 1

# Typical_total_federal_loan_debt_by_family_income.to_csv('../../user-notebooks/yjin/USNews/results/Student Loan Debt by family income(Typical total).csv')
# Undergraduates_federal_loan_debt_by_family_income.to_csv('../../user-notebooks/yjin/USNews/results/Student Loan Debt by family income(Undergraduates).csv')
# general_df.to_csv('../../user-notebooks/yjin/USNews/results/Student Loan Debt(general).csv')

# Campus

Log in and scrape campus regular info

In [ ]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                         ' Chrome/84.0.4147.105 Safari/537.36',
           'username': 'your username',
           'password': 'your password',
           # you may need to change cookie here:
           # log in on your browser, find it in the 'Network' section, 'Request Headers' block, with developer tool
           "cookie": 'get with developer tool'}
regular_info = pd.DataFrame(index=Urls['University'])

for i in range(len(Urls)):
    url = ('https://www.usnews.com' + Urls.iloc[i, 1] + '/campus-info').replace(' ', '')
    print(url)

    session = HTMLSession()
    page = session.get(url, headers=headers, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')

    body = soup.find_all('div', class_='Display__BellowStyled-h3gn08-6 gdKeTK')

    # get main sections title:
    ID = []
    for section in body:
        ID.append(section['id'])

    # map main sections and get info inside
    for j in range(len(ID)):
        k = 1
        while True:
            test = soup.select('#{0} > div > div > div:nth-child({1})'.format(ID[j], k))
            if len(test) == 0:  # which means there's no next block in this section
                break  # therefore come to the next section
            test_children = list(test[0].children)
            aa = test[0].find_all('div', recursive=False)

            for x in aa:
                temp = []
                for y in x.strings:
                    temp.append(y)
                if 'Jump To Section:' not in temp:
                    col_name = ' - '.join([ID[j], temp[0]])
                    if col_name not in regular_info.columns:  # create a new column if the category doesn't exist
                        regular_info[col_name] = 'N/A'
                    if len(temp) == 2:
                        regular_info[col_name][i] = temp[1]
                    else:
                        regular_info[col_name][i] = ', '.join(temp[1:])
                    k += 1
                else:  # if it's a button div which doesn't include any useful info, we skip
                    k += 1
                    break

# regular_info.to_csv('../../user-notebooks/yjin/USNews/results/general info in campus block.csv')

## Links of Images shown in Campus

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from requests_html import HTMLSession
import re
import time

Functions to help mimicing user actions:

In [ ]:
def window_handler(brs):
    try:
        s_window = brs.find_element_by_xpath("/html/body/div[3]/div/div/button")
        s_window.click()
    except:
        pass

    try:
        s_window = brs.find_element_by_xpath("/html/body/div[5]/div/div/button")
        s_window.click()
    except:
        pass

    try:
        s_window = brs.find_element_by_xpath('/html/body/div[6]/div/div/button')
        s_window.click()
    except:
        pass


def window_handler2(brs):
    try:
        s_window = brs.find_element_by_xpath("/html/body/div[3]/div/div/button")
        s_window.click()
    except:
        pass

    try:
        s_window = brs.find_element_by_xpath("/html/body/div[5]/div/div/button")
        s_window.click()
    except:
        pass

    try:
        s_window = brs.find_element_by_css_selector(
            '#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > div > div.Generic__ContentBox-sc-1itie0w-0.cPiuYB.content > div.section-box > div.mt6 > react-trigger > div > div.Slideshow__Container-bw1gs6-0.gCuKqa > div.Arrows__Container-sc-1s7bkvt-2.jAYcej > button:nth-child(3)')
        s_window.click()
    except:
        pass

    try:
        s_window = brs.find_element_by_xpath('/html/body/div[6]/div/div/button')
        s_window.click()
    except:
        pass


Main part of the scrapy:

In [ ]:
Urls = pd.read_csv('material/urls.csv', names=['University', 'URL'])
options = Options()
options.add_argument("--disable-notifications")

session = HTMLSession()


img_df = pd.DataFrame(index=Urls['University'])
for i in range(24):
    col_name = 'Img_{0}'.format(i)
    img_df[col_name] = 'N/A'
record_df = pd.read_csv('campus/image count.csv', index_col=0)
img_df = pd.read_csv('campus/image links new 8.csv', index_col=0).fillna('N/A')


for i in range(1, len(Urls)):
    url = ('https://www.usnews.com' + Urls.iloc[i, 1] + '/campus-info').replace(' ', '')
    browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    browser.get(url)

    content = session.get(url)
    soup = BeautifulSoup(browser.page_source, "html.parser")

    basic_address = "#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > div > " \
                    "div > div.section-box > div.mt6"
    results_0 = content.html.find(basic_address)

    if len(results_0) > 0:
        text = results_0[0].text
        a = re.search('\d.*of.*\d', text)
        N = int(a.group().split(' of ')[1].split('\n')[0])
        record_df.iloc[i, 0] = N

        # if numbers of pictures are more then previous setting
        if N > img_df.shape[1]:
            for n in range(img_df.shape[1], N):
                col_name = 'Img_{0}'.format(n)
                img_df[col_name] = 'N/A'

        while True:
            try:
                element1 = soup.find_all("img", {"class": "SlideshowInContent__SlideImage-ecnkfw-2 tSQUx"})[0]
            except:
                try:
                    window_handler(browser)
                    soup = BeautifulSoup(browser.page_source, "html.parser")
                except:
                    window_handler(browser)
                    soup = BeautifulSoup(browser.page_source, "html.parser")
            else:
                break

        try:
            link = element1['src']
        except:
            link = 'EMPTY'
        img_df.iloc[i, 0] = link

        # map all images
        n = 1
        if N >= 2:
            while img_df.iloc[i, N - 1] == 'N/A':
                try:
                    page_next = browser.find_element_by_css_selector(
                        '#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > '
                        'div > div > div.section-box > div.mt6 > react-trigger > div > '
                        'div.Slideshow__Container-bw1gs6-0.gCuKqa > div.Arrows__Container-sc-1s7bkvt-2.jAYcej > '
                        'button:nth-child(3)')
                    page_next.click()
                except:
                    # time.sleep(2)
                    window_handler(browser)
                    soup = BeautifulSoup(browser.page_source, "html.parser")
                    count = browser.find_element_by_css_selector(
                        '#app > div > div:nth-child(1) > div:nth-child(8) > div > '
                        'div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > div > '
                        'div.Generic__ContentBox-sc-1itie0w-0.cPiuYB.content > '
                        'div.section-box > div.mt6 > react-trigger > div > '
                        'div.Slideshow__Container-bw1gs6-0.gCuKqa > '
                        'div.Slideshow__SlideContainer-bw1gs6-1.kcTrLh > div > span')
                    compare = count.text.split(' of ')
                    print(compare)
                    if compare[0] != compare[1] and int(compare[0]) <= n:
                        print('mark 8-1')
                        page_next = browser.find_element_by_css_selector(
                            '#app > div > div:nth-child(1) > div:nth-child(8) > div > div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > '
                            'div > div > div.section-box > div.mt6 > react-trigger > div > '
                            'div.Slideshow__Container-bw1gs6-0.gCuKqa > div.Arrows__Container-sc-1s7bkvt-2.jAYcej > '
                            'button:nth-child(3)')
                        print('mark 8-2')
                        location = page_next.location
                        size = page_next.size
                        print('mark 8-3')
                        browser.execute_script("window.scrollTo({0}, {1})".format(0, location['y']-size['height']*2))
                        print('mark 8-4')
                        while True:
                            try:
                                page_next.click()
                            except:
                                window_handler(browser)
                            else:
                                break
                    else:
                        print('mark 8-break')
                        break

                print("link and ...")
                while True:
                    try:
                        soup = BeautifulSoup(browser.page_source, "html.parser")
                        element1 = \
                            soup.find_all("img", {"class": "SlideshowInContent__SlideImage-ecnkfw-2 tSQUx"})[0]
                        count = browser.find_element_by_css_selector(
                            '#app > div > div:nth-child(1) > div:nth-child(8) > div > '
                            'div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > div > '
                            'div.Generic__ContentBox-sc-1itie0w-0.cPiuYB.content > '
                            'div.section-box > div.mt6 > react-trigger > div > '
                            'div.Slideshow__Container-bw1gs6-0.gCuKqa > '
                            'div.Slideshow__SlideContainer-bw1gs6-1.kcTrLh > div > span')
                        n = int(count.text.split(' of ')[0]) - 1
                    except:
                        try:
                            window_handler(browser)
                            soup = BeautifulSoup(browser.page_source, "html.parser")
                            element1 = \
                                soup.find_all("img", {"class": "SlideshowInContent__SlideImage-ecnkfw-2 tSQUx"})[0]
                            count = browser.find_element_by_css_selector(
                                '#app > div > div:nth-child(1) > div:nth-child(8) > div > '
                                'div.Cell-sc-1abjmm4-0.kXBDnR.mb0 > div > '
                                'div.Generic__ContentBox-sc-1itie0w-0.cPiuYB.content > '
                                'div.section-box > div.mt6 > react-trigger > div > '
                                'div.Slideshow__Container-bw1gs6-0.gCuKqa > '
                                'div.Slideshow__SlideContainer-bw1gs6-1.kcTrLh > div > span')
                            n = int(count.text.split(' of ')[0]) - 1
                        except:
                            window_handler2(browser)
                    else:
                        break

                try:
                    link = element1['src']
                except:
                    link = 'EMPTY'
                if img_df.iloc[i, n] == 'N/A':
                    img_df.iloc[i, n] = link
                print('Time {0} finished'.format(n))
                n += 1

    browser.close()


# img_df.to_csv('../../user-notebooks/yjin/USNews/results/image links.csv')
